In [12]:
from langchain_community.retrievers import BM25Retriever
from typing import List
import jieba  # 分词方法
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 循环字节分块
from langchain.vectorstores import FAISS, Qdrant  # 向量数据库
from langchain.document_loaders import TextLoader  # 加载文本的加载器
from langchain_huggingface import HuggingFaceEmbeddings  # type: ignore # 向量器

加载数据

In [13]:
loader = TextLoader("drug_100.txt")
document = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 每个分块的大小
    chunk_overlap=0,  # 分块之间的重叠部分
    length_function=len,  # 使用长度函数来计算分块大小
    separators=["\n"],  # 分块的分隔符
)
docs = text_splitter.split_documents(document)  # 分割文档为多个分块,以列表形式返回

In [14]:
docs[0]

Document(metadata={'source': 'drug_100.txt'}, page_content='"氨肽素\\n药品类型:血液系统药物，促进白细胞增生药\\n英文名:Ampeptide Elemente\\n剂型:片剂：0.2g\\n适应证:1.适用于治疗特发性血小板减少性紫癜(原发性血小板减少性紫癜)、过敏性紫癜、慢性白细胞减少症、慢性再生障碍性贫血。2.对银屑病也有一定疗效。\\n禁忌证:尚不明确。\\n注意事项:少数患者于停药后可能出现反跳现象，但继续用药仍可有效。\\n不良反应:个别病例服用后有腹部不适。\\n用法用量:口服给药：每次0.1g，每天3次。用药至少4周，有效者可连续服用。\\n药物相互作用:尚不明确。",\n"利可君\\n药品类型:血液系统药物，促进白细胞增生药\\n英文名:Leucogen\\n别名:利血生\\n剂型:片剂：10mg，20mg。\\n适应证:用于防治各种原因引起的白细胞减少、再生障碍性贫血等。\\n禁忌证:尚不明确。\\n注意事项:使用本药剂量要适当，过高或过低均影响疗效。\\n不良反应:本药无不良反应报道。\\n用法用量:口服给药：1.成人：每次10～20mg，每天3次，疗程1个月。2.儿童：每次10mg，每天2～3次，疗程1个月。\\n药物相互作用:尚不明确。",')

使用BM25方式检索

In [15]:
def preprocessing_func(text: str) -> List[str]:
    """
    分词函数，将文本分词为列表
    """
    return list(jieba.cut(text))  # 使用jieba进行中文分词

bm25 = BM25Retriever(
    docs=docs, # 文档列表
    k = 5,  # 返回的文档数量
)
print(bm25.k) # 输出BM25检索器的k值

# 原始的bm25算法分词是使用的英文分词法, 所以需要去适配中文的分词策略,使用jieba进行中文分词
# from_documents()方式是用于->自动化处理文档的分词和索引构建​​
retriever = bm25.from_documents(
    docs,
    preprocess_func=preprocessing_func,  # 使用自定义的分词函数
    k=5,  # 返回的文档数量
)

5


测试

In [16]:
retriever.invoke("内分泌药物有哪些, 作用是什么？")  # 调用检索器进行查询

[Document(metadata={'source': 'drug_100.txt'}, page_content='\n"去氨加压素\\n药品类型:内分泌系统药物，下丘脑及影响内分泌的药物\\n英文名:Desmopressin\\n别名:的斯加压素；弥凝鼻；Desmospray；Minirin\\n剂型:1.鼻喷雾剂（Desmospray）：500μg(5ml)；2.鼻滴入剂：200μg(2ml)；3.片剂：0.1mg，0.2mg；4.注射剂：0.4μg(1ml)。\\n适应证:1.轻中度血友病甲、血管性血友病1型和2型(2B型除外)。2.因尿毒症、肝硬化或药物诱发出血(如肝素、阿司匹林和右旋糖酐等)，其他血小板功能障碍和不明原因引起的出血时间过长。3.心脏手术引起的出血时间过长。4.其他手术引起的出血时间过长。不过，本品对重症血友病甲疗效较差。5.尿崩症，由于本品作用时间较长并可鼻腔或口服给药，因此是治疗尿崩症的首选药物。6.小儿遗尿。\\n禁忌证:1.对本品过敏者、哺乳者禁用。2.烦渴症、不稳定型心绞痛和严重充血性心衰患者禁用。3.禁用于2B型遗传性假性血友病患者。4.孕妇分娩前禁用；如有必要，分娩后立即给药。\\n注意事项:1.有血栓形成倾向、冠心病、高血压者慎用。2.治疗期间需限制饮水。3.老年人及幼儿、有颅内压增高趋势的患者及水电解质紊乱的患者慎用。4.需服利尿剂的患者、心功能不全、HB型血友病、不稳定性心绞痛等患者忌用。5.用药过量会增加水潴留和低钠血症的危险。对已出现低钠血症的患者，应停药、限制摄入量，伴有症状的患者，可输注等渗或高渗氯化钠注射剂。6.水钠潴留严重者可使用呋噻米利尿。\\n不良反应:不常见。1.偶有短暂头痛、恶心、轻度腹痛、血压升高和心率增快，减少剂量常可消失。2.少数患者肌内注射局部可出现疼痛。长期鼻黏膜给药可发生鼻黏膜充血、鼻炎、鼻黏膜萎缩，影响药物吸收。过量会引起水中毒和低钠血症。3.使用大剂量时，会产生头痛、一过性血压下降、反射性心率加速、面红、胃痛和恶心。\\n用法用量:1.中枢性尿崩症替代治疗：(1)喷雾或滴入制剂鼻腔给药：每次5～10μg，每天1～2次；(2)口服片剂给药：每次0.1～0.2mg(100～200μg)，每天3次；(3)肌内注射，每次1～4μg，每天1～2次。由于各个患者的病情轻重不同，用药应从小剂量开

In [17]:
from rank_bm25 import BM25Okapi
texts = [i.page_content for i in docs]  # 提取文档内容
texts_processed = [preprocessing_func(text) for text in texts]  # 对每个文档chunk进行分词处理,得到所有的词表(用于索引所有的单词)

# print(texts_processed)  # 输出第一个文档chunk的分词结果


In [18]:
vectorizer = BM25Okapi(texts_processed)  # 创建BM25向量化器
vectorizer.get_top_n(preprocessing_func("内分泌药物有哪些, 作用是什么？"), texts, n=5)  # 获取与查询最相关的前5个文档, 比较“骨折了怎么办”与texts的BM25相似度

['\n"去氨加压素\\n药品类型:内分泌系统药物，下丘脑及影响内分泌的药物\\n英文名:Desmopressin\\n别名:的斯加压素；弥凝鼻；Desmospray；Minirin\\n剂型:1.鼻喷雾剂（Desmospray）：500μg(5ml)；2.鼻滴入剂：200μg(2ml)；3.片剂：0.1mg，0.2mg；4.注射剂：0.4μg(1ml)。\\n适应证:1.轻中度血友病甲、血管性血友病1型和2型(2B型除外)。2.因尿毒症、肝硬化或药物诱发出血(如肝素、阿司匹林和右旋糖酐等)，其他血小板功能障碍和不明原因引起的出血时间过长。3.心脏手术引起的出血时间过长。4.其他手术引起的出血时间过长。不过，本品对重症血友病甲疗效较差。5.尿崩症，由于本品作用时间较长并可鼻腔或口服给药，因此是治疗尿崩症的首选药物。6.小儿遗尿。\\n禁忌证:1.对本品过敏者、哺乳者禁用。2.烦渴症、不稳定型心绞痛和严重充血性心衰患者禁用。3.禁用于2B型遗传性假性血友病患者。4.孕妇分娩前禁用；如有必要，分娩后立即给药。\\n注意事项:1.有血栓形成倾向、冠心病、高血压者慎用。2.治疗期间需限制饮水。3.老年人及幼儿、有颅内压增高趋势的患者及水电解质紊乱的患者慎用。4.需服利尿剂的患者、心功能不全、HB型血友病、不稳定性心绞痛等患者忌用。5.用药过量会增加水潴留和低钠血症的危险。对已出现低钠血症的患者，应停药、限制摄入量，伴有症状的患者，可输注等渗或高渗氯化钠注射剂。6.水钠潴留严重者可使用呋噻米利尿。\\n不良反应:不常见。1.偶有短暂头痛、恶心、轻度腹痛、血压升高和心率增快，减少剂量常可消失。2.少数患者肌内注射局部可出现疼痛。长期鼻黏膜给药可发生鼻黏膜充血、鼻炎、鼻黏膜萎缩，影响药物吸收。过量会引起水中毒和低钠血症。3.使用大剂量时，会产生头痛、一过性血压下降、反射性心率加速、面红、胃痛和恶心。\\n用法用量:1.中枢性尿崩症替代治疗：(1)喷雾或滴入制剂鼻腔给药：每次5～10μg，每天1～2次；(2)口服片剂给药：每次0.1～0.2mg(100～200μg)，每天3次；(3)肌内注射，每次1～4μg，每天1～2次。由于各个患者的病情轻重不同，用药应从小剂量开始，根据患者反应来调整药量。2.轻至中等的甲型血友病治疗，一般以0.3～0.4μg/kg(300～400ng/kg)进

使用embedding向量进行检索

In [19]:
from huggingface_hub import snapshot_download
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"   # 使用国内镜像
model_path = snapshot_download(repo_id="BAAI/bge-m3", repo_type="model")  # 下载模型

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [20]:
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-zh-v1.5",  # 使用BGE模型
    model_kwargs={"device": "cuda"},  # 使用gpu
)

需要提前部署Qdrant的api
```bash
wget https://github.com/qdrant/qdrant/releases/download/v1.14.1/qdrant-x86_64-unknown-linux-musl.tar.gz
tar -xzf qdrant-x86_64-unknown-linux-musl.tar.gz
./qdrant
```

In [21]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)

try:
    collections = client.get_collections()  # 测试连接
    print("连接成功！", collections)
except Exception as e:
    print("连接失败:", e)

连接成功！ collections=[CollectionDescription(name='bff42a2a790142c697b4b43e64e5033d'), CollectionDescription(name='f85c5a1ab31c4d02ba08b1d1c35945e9')]


使用FAISS实现向量生成 

from_documents()方式是用于->自动化处理文档的分词和索引构建​​, docs是表示需要转换的文档, embeddings表示去使用的embeddings向量

In [31]:
db = Qdrant.from_documents(
    docs,
    embeddings,
)

In [32]:
bm25res = vectorizer.get_top_n(
    preprocessing_func("内分泌药物有哪些, 作用是什么？"),  # 查询文本
    texts,  # 文档列表
    n=5,  # 返回的文档数量
)
 
bm25res

['\n"去氨加压素\\n药品类型:内分泌系统药物，下丘脑及影响内分泌的药物\\n英文名:Desmopressin\\n别名:的斯加压素；弥凝鼻；Desmospray；Minirin\\n剂型:1.鼻喷雾剂（Desmospray）：500μg(5ml)；2.鼻滴入剂：200μg(2ml)；3.片剂：0.1mg，0.2mg；4.注射剂：0.4μg(1ml)。\\n适应证:1.轻中度血友病甲、血管性血友病1型和2型(2B型除外)。2.因尿毒症、肝硬化或药物诱发出血(如肝素、阿司匹林和右旋糖酐等)，其他血小板功能障碍和不明原因引起的出血时间过长。3.心脏手术引起的出血时间过长。4.其他手术引起的出血时间过长。不过，本品对重症血友病甲疗效较差。5.尿崩症，由于本品作用时间较长并可鼻腔或口服给药，因此是治疗尿崩症的首选药物。6.小儿遗尿。\\n禁忌证:1.对本品过敏者、哺乳者禁用。2.烦渴症、不稳定型心绞痛和严重充血性心衰患者禁用。3.禁用于2B型遗传性假性血友病患者。4.孕妇分娩前禁用；如有必要，分娩后立即给药。\\n注意事项:1.有血栓形成倾向、冠心病、高血压者慎用。2.治疗期间需限制饮水。3.老年人及幼儿、有颅内压增高趋势的患者及水电解质紊乱的患者慎用。4.需服利尿剂的患者、心功能不全、HB型血友病、不稳定性心绞痛等患者忌用。5.用药过量会增加水潴留和低钠血症的危险。对已出现低钠血症的患者，应停药、限制摄入量，伴有症状的患者，可输注等渗或高渗氯化钠注射剂。6.水钠潴留严重者可使用呋噻米利尿。\\n不良反应:不常见。1.偶有短暂头痛、恶心、轻度腹痛、血压升高和心率增快，减少剂量常可消失。2.少数患者肌内注射局部可出现疼痛。长期鼻黏膜给药可发生鼻黏膜充血、鼻炎、鼻黏膜萎缩，影响药物吸收。过量会引起水中毒和低钠血症。3.使用大剂量时，会产生头痛、一过性血压下降、反射性心率加速、面红、胃痛和恶心。\\n用法用量:1.中枢性尿崩症替代治疗：(1)喷雾或滴入制剂鼻腔给药：每次5～10μg，每天1～2次；(2)口服片剂给药：每次0.1～0.2mg(100～200μg)，每天3次；(3)肌内注射，每次1～4μg，每天1～2次。由于各个患者的病情轻重不同，用药应从小剂量开始，根据患者反应来调整药量。2.轻至中等的甲型血友病治疗，一般以0.3～0.4μg/kg(300～400ng/kg)进

In [24]:
vector_res = db.similarity_search(
    "内分泌药物有哪些, 作用是什么？",  # 查询文本
    k=5,  # 返回的文档数量
)
vector_res


[Document(metadata={'source': 'drug_100.txt', '_id': 'fe153cff-8cb5-44aa-9a92-4a7b1abdbce2', '_collection_name': '74da3d6823294db98395ae62367afafc'}, page_content='\n"达那唑\\n药品类型:内分泌系统药物，性腺疾病用药\\n英文名:Danazol\\n别名:安宫唑；丹那唑；炔睾醇；Danatrol；Danazolum；Danocrine；Danol\\n剂型:胶囊：100mg，200mg。\\n适应证:1.主要用于对其他药物治疗不能耐受或治疗无效的子宫内膜异位。2.尚可用于治疗乳腺囊性增生病(纤维囊性乳腺病)、男性乳腺发育症、乳腺痛、痛经等。3.还可用于特发性血小板减少性紫瘢、遗传性血管性水肿、系统性红斑狼疮、青春期性早熟、不孕症与血友病及血友病乙(Christmas病)等。\\n禁忌证:1.严重心、肝、肾功能不全。2.原因不明的生殖器官出血。3.哺乳期妇女。4.癫痫及严重高血压患者禁用。\\n注意事项:1.慎用：因本药可引起一定程度的体液潴留，故有癫痫、偏头痛或心肾功能不全者应慎用，必须严密监护。2.药物对妊娠的影响：治疗期间一般不会妊娠，一旦发现妊娠，应立即停药。理论上达那唑对女性胎儿可能有雌激素的效应，但临床上极少发现。3.本药可影响糖耐量试验及甲状腺功能试验的结果，也可使血清总T4降低，血清T3增高。4.使用本药时应注意有无心、肝、肾功能损害及生殖器官出血，对男性应注意睾丸大小。男性用药时，须随访精液量及黏度，并进行精子计数与检测精子活动力，建议每3至4个月查1次，特别是对青年患者。5.对原因不明的男性乳房发育，在手术前可考虑先用本药治疗。6.对青春期性早熟，本药能使患者月经停止，乳房发育退化；由于有增加骨成长的刺激作用，较其他治疗性早熟药物无明显优点，故仅限于对其他药物治疗无效的重度患者使用。7.如停药已60～90天，仍无规则月经，则应进行诊治。服药期间需避孕者，应采用非甾体激素的避孕方法，不用口服避孕药。8.女性用药如果出现男性化症状，应停止达那唑治疗。\\n不良反应:1.较常见的不良反应女性为闭经，突破性出血和不规则阴道出血，并可有乳房缩小、声音嘶哑、毛发增多等；无论男女，均可出现粉刺、皮

对文本召回和向量召回的方法进行**重排序**

In [33]:
def rrf(vector_result: List[str], bm25_result: List[str], k: int = 10, m: int = 60) -> List[str]:
    """
    RRF (Reciprocal Rank Fusion) 算法融合向量检索和BM25检索结果
    param:
        vector_result: 向量检索结果列表
        bm25_result: BM25检索结果列表
        k: 排序后返回前k个文档
        m: 用于倒数排名加权的参数
    """
    # 创建一个字典来存储每个文档的分数
    doc_scores = {}

    # 遍历两组结果, 计算每个文档的融合得分
    for rank, doc_id in enumerate(vector_result):
        doc_scores[doc_id] = doc_scores.get(doc_id, 0) + 1 / (rank + m)  # 向量检索结果的倒数排名加权
    
    for rank, doc_id in enumerate(bm25_result):
        doc_scores[doc_id] = doc_scores.get(doc_id, 0) + 1 / (rank + m)  # BM25检索结果的倒数排名加权
    
    # 按照分数排序并返回前k个文档
    sorted_result = [d for d, _ in sorted(doc_scores.items(), key=lambda item: item[1], reverse=True)[:k]]

    return sorted_result  # 返回前k个文档内容


In [34]:
vector_result = [i.page_content for i in vector_res]  # 提取向量检索结果的文档内容
text_result = [i for i in bm25res]  # 提取BM25检索结果的文档内容

rrf_result = rrf(vector_result, text_result, k=5, m=60)  # 使用RRF算法融合结果
rrf_result

['\n"司坦唑醇\\n药品类型:内分泌系统药物，性腺疾病用药\\n英文名:Stanozolol\\n别名:康力龙；吡唑甲氢龙；吡唑甲基睾丸素；司坦唑；Androstanazole；Terabolin\\n剂型:片剂：2mg。\\n适应证:用于慢性消耗性疾病、骨质疏松、重病及手术后体弱消瘦、年老体弱、小儿发育不良、再生障碍性贫血、白细胞减少症、血小板减少症、高脂血症等。用于年老多病、产后衰弱等。还用于防治长期使用皮质激素引起的肾上腺皮质功能减退症。\\n禁忌证:前列腺肥大、前列腺癌患者及孕妇禁用。\\n注意事项:1.患胃溃疡，肝、肺、心等功能不全者慎用。2.如长期使用可有肝功能障碍、黄疸等。3.如出现痤疮等男性化反应应停药。\\n不良反应:连续应用3～5个月，未有明显的雄性化作用，个别青年女患者有月经推迟现象，停药即复原。服药初期可能下肢、颜面出现水肿，继续服药能自行消失。出现男性化不良反应，建议停药。\\n用法用量:口服：成人每天4～6mg，分2～3次；小儿每天2～4mg，分1～3次口服。再生障碍性贫血用量较大且时间较长时，须由医师掌握，一般最大量可用到每天12mg，分3次口服，要注意肝功能损害情况及保肝治疗。\\n药物相互作用:巴比妥类、卡马西平、甲丙氨酯、保泰松、利福平等会减低雌激素活性。",',
 '\n"达那唑\\n药品类型:内分泌系统药物，性腺疾病用药\\n英文名:Danazol\\n别名:安宫唑；丹那唑；炔睾醇；Danatrol；Danazolum；Danocrine；Danol\\n剂型:胶囊：100mg，200mg。\\n适应证:1.主要用于对其他药物治疗不能耐受或治疗无效的子宫内膜异位。2.尚可用于治疗乳腺囊性增生病(纤维囊性乳腺病)、男性乳腺发育症、乳腺痛、痛经等。3.还可用于特发性血小板减少性紫瘢、遗传性血管性水肿、系统性红斑狼疮、青春期性早熟、不孕症与血友病及血友病乙(Christmas病)等。\\n禁忌证:1.严重心、肝、肾功能不全。2.原因不明的生殖器官出血。3.哺乳期妇女。4.癫痫及严重高血压患者禁用。\\n注意事项:1.慎用：因本药可引起一定程度的体液潴留，故有癫痫、偏头痛或心肾功能不全者应慎用，必须严密监护。2.药物对妊娠的影响：治疗期间一般不会妊娠，一旦发现妊娠，应立即停药。理论上达那唑对女性胎儿可能有雌激素的效应，但临床上极少

使用LLM + RAG的方式进行召回

In [35]:
prompt = '''
    任务目标：根据检索出的文档回答用户问题
    任务要求：
    1. 只使用检索出的文档内容回答问题
    2. 不要添加任何额外的内容
    3. 如果检索结果中没有相关信息，请回答“无法回答”

用户问题:
{}

检索的文档:
{}

'''

In [36]:
from langchain_openai import ChatOpenAI  # OpenAI的聊天模型
model = ChatOpenAI(
    model="Qwen3-4B",
    base_url="http://localhost:8000/v1",  # 本地部署的Qwen3模型地址
    api_key="your_api_key",  # 替换为你的API密钥,
    max_tokens=1000,  # 最大生成的token数量
)

手动阶段模型输出的最大长度

In [41]:
def truncate_text(text, max_chars=4096):
    return text[:max_chars] if len(text) > max_chars else text


In [42]:
input_prompt = prompt.format(
        "内分泌药物有哪些, 作用是什么？",  # 用户问题
        "\n".join(rrf_result)  # 将融合后的文档内容作为检索结果传入
    )
input_prompt = truncate_text(input_prompt)  # 截断输入文本以适应模型的最大长度

采用RAG检索增强生成的模型回答

In [43]:
res = model.invoke(
    input_prompt,  # 输入文本
)

print(res.content)  # 输出模型的回答结果



内分泌药物包括司坦唑醇、达那唑和去氨加压素。  
- **司坦唑醇**：用于慢性消耗性疾病、骨质疏松、术后体弱消瘦、再生障碍性贫血、高脂血症等。  
- **达那唑**：用于子宫内膜异位、乳腺囊性增生、男性乳腺发育、痛经、血小板减少性紫癜等。  
- **去氨加压素**：用于尿崩症、血友病甲、血管性血友病、小儿遗尿等。


原始模型的回答

In [44]:
res = model.invoke("内分泌药物有哪些, 作用是什么？")
print(res.content)  # 输出模型的回答结果



内分泌药物是用于调节内分泌系统功能的药物，主要通过影响激素的分泌、释放、作用或代谢来治疗相关疾病。以下是常见的内分泌药物分类及其作用：

---

### **一、甲状腺相关药物**
1. **甲状腺激素替代药物**  
   - **药物**：左甲状腺素钠（优甲乐）、甲状腺粉  
   - **作用**：补充或替代甲状腺激素（T3、T4），用于治疗甲状腺功能减退症（甲减）。  
   - **适应症**：甲状腺手术后、甲状腺炎、甲状腺结节等导致的激素不足。

2. **抗甲状腺药物**  
   - **药物**：甲巯咪唑（他巴唑）、丙硫氧嘧啶  
   - **作用**：抑制甲状腺激素合成，用于治疗甲状腺功能亢进症（甲亢）。  
   - **适应症**：Graves病、毒性结节性甲状腺肿等。

---

### **二、肾上腺相关药物**
1. **糖皮质激素**  
   - **药物**：泼尼松、地塞米松、氢化可的松  
   - **作用**：抗炎、免疫抑制、抗过敏，调节代谢和应激反应。  
   - **适应症**：自身免疫性疾病（如类风湿性关节炎）、炎症性疾病（如哮喘）、肾上腺皮质功能减退等。

2. **盐皮质激素**  
   - **药物**：氟氢可的松  
   - **作用**：调节钠、钾平衡，维持血压和体液稳定。  
   - **适应症**：原发性醛固酮增多症（高血压患者）。

---

### **三、胰岛相关药物**
1. **胰岛素类药物**  
   - **药物**：人胰岛素、胰岛素类似物（速效、中效、长效）  
   - **作用**：调节血糖，促进葡萄糖
